In [ ]:
'''Import libraries (tensorflow and time)'''

import tensorflow as tf
import time

In [2]:
'''List physical devices in the notebook'''

devices = tf.config.list_physical_devices()
for device in devices:
  print(device)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
'''The heavy computational task used is matrix multiplication'''
#the function returns the total time needed to execute the task

def compute_on_device(device_name):
  with tf.device(device_name):
    start_time = time.time() #mark the start time

    #generate random matrices in size 1000x1000
    matrix_size = 1000
    a = tf.random.normal([matrix_size, matrix_size])
    b = tf.random.normal([matrix_size, matrix_size])

    #multiply the two matrices
    result = tf.matmul(a, b)

    end_time = time.time() #mark the end time
  return end_time - start_time

In [4]:
'''The first load balancing algorithm: Round Robin'''
#assume that the total number of tasks is 10
#the tasks are sequentially distributed across both devices

def round_robin():
  devices = ['/CPU:0', '/GPU:0'] #list the 2 devices
  num_tasks = 10

  for i in range(num_tasks):
    device_used = devices[i % len(devices)]
    task_time = compute_on_device(device_used)
    print(f"Task {i + 1} executed on device {device_used}, Time: {task_time:.4f}")

#call the function
round_robin()

Task 1 executed on device /CPU:0, Time: 0.1837
Task 2 executed on device /GPU:0, Time: 0.1558
Task 3 executed on device /CPU:0, Time: 0.0651
Task 4 executed on device /GPU:0, Time: 0.0017
Task 5 executed on device /CPU:0, Time: 0.0604
Task 6 executed on device /GPU:0, Time: 0.0014
Task 7 executed on device /CPU:0, Time: 0.0543
Task 8 executed on device /GPU:0, Time: 0.0015
Task 9 executed on device /CPU:0, Time: 0.0531
Task 10 executed on device /GPU:0, Time: 0.0013


In [5]:
'''The second load balancing algorithm: Weighted Round Robin'''
#assume that CPU capacity is 7, while GPU capacity is 3
#assume that both devices start with zero tasks
#assume that the total number of tasks is 10

def weighted_round_robin():
  devices_capacity = {"/CPU:0":7, "/GPU:0":3} #capacity of CPU=7, capacity of GPU=3
  devices = {"/CPU:0":0, "/GPU:0":0} #devices start at zero tasks
  num_tasks = 10

  for i in range(num_tasks):
    if devices["/CPU:0"] < devices_capacity["/CPU:0"]: #if the number of tasks for CPU is less than its capacity
      device_used = "/CPU:0" #then use CPU
    else:
      device_used = "/GPU:0" #else use GPU

    task_time = compute_on_device(device_used)
    devices[device_used] += 1 #increment the number of tasks for the used device
    print(f"Task {i + 1} executed on device {device_used}, Time: {task_time:.4f}")

#call the function
weighted_round_robin()

Task 1 executed on device /CPU:0, Time: 0.0581
Task 2 executed on device /CPU:0, Time: 0.0517
Task 3 executed on device /CPU:0, Time: 0.0589
Task 4 executed on device /CPU:0, Time: 0.0545
Task 5 executed on device /CPU:0, Time: 0.0543
Task 6 executed on device /CPU:0, Time: 0.0517
Task 7 executed on device /CPU:0, Time: 0.0567
Task 8 executed on device /GPU:0, Time: 0.0014
Task 9 executed on device /GPU:0, Time: 0.0009
Task 10 executed on device /GPU:0, Time: 0.0009


In [7]:
'''The third load balancing algorithm: Least Connections'''
#assume that CPU has three current tasks while GPU has 1 current task
#assume that the total number of tasks is 10

def least_connections():
  devices = {"/CPU:0":3, "/GPU:0":1} #current tasks for both devices
  num_tasks = 10

  for i in range(num_tasks):
    device_used = min(devices, key = devices.get) #use the device with the minimum number of tasks
    task_time = compute_on_device(device_used)
    devices[device_used] += 1 #increment the number of tasks for the used device
    print(f"Task {i + 1} executed on device {device_used}, Time: {task_time:.4f}")

#call the function
least_connections()

Task 1 executed on device /GPU:0, Time: 0.0025
Task 2 executed on device /GPU:0, Time: 0.0020
Task 3 executed on device /CPU:0, Time: 0.0533
Task 4 executed on device /GPU:0, Time: 0.0012
Task 5 executed on device /CPU:0, Time: 0.0542
Task 6 executed on device /GPU:0, Time: 0.0012
Task 7 executed on device /CPU:0, Time: 0.0565
Task 8 executed on device /GPU:0, Time: 0.0020
Task 9 executed on device /CPU:0, Time: 0.0655
Task 10 executed on device /GPU:0, Time: 0.0015


# Report

## *   How does each algorithm distribute the tasks?

1.   Round Robin

This algorithm distributes the tasks sequentially and evenly on all devices.

2.   Weighted Round Robin

Each device has a capacity on the amount of tasks it can handle at once. This algorithm makes sure that no device exceeds its capacity and crashes, therefore it ensures weighted distribution.

3.   Least Connections

This algorithm gives the task to the server with the least tasks, or least connections.

## *   Performance comparision for each algorithm


1.   Round Robin

For the first 2 tasks (the first task for each device), both devices start slow with GPU being slightly faster than CPU. Then for the rest of the tasks, both devices become faster with GPU being slightly faster than CPU.


2.   Weighted Round Robin

CPU took the first 7 tasks reaching its capacity before GPU took over the left 3 tasks. GPU is definitely faster than CPU.

3. Least Connections

GPU started with the tasks until it equaled CPU with its tasks. GPU is also faster than CPU.



